In [ ]:
!pip install openai==0.28 python-dotenv
import os

import openai

from dotenv import load_dotenv

load_dotenv()



# Set up your OpenAI API key
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [ ]:
def classify_text_improved(input_text, model="gpt-3.5-turbo-instruct"):

    # Step 1: Identify Clues
    clues_prompt = (
        f"This is an overall sentiment classifier.\n"
        f"First, list CLUES (i.e., keywords, phrases, contextual information, semantic relations, "
        f"semantic meaning, tones, references) that support the sentiment determination of the input.\n"
        f"Input: {input_text}\n"
        f"Clues:"
    )

    response_clues = openai.Completion.create(
        model=model,
        prompt=clues_prompt,
        max_tokens=100,
        temperature=0.7
    )

    clues = response_clues.choices[0].text.strip()
    print("Clues Response:", response_clues.choices[0].text)  # Print Clues Response

    # Step 2: Diagnostic Reasoning
    reasoning_prompt = (
        f"Based on the input and clues, deduce the diagnostic reasoning process from premises "
        f"(i.e., clues and input) that supports the sentiment determination of the input.\n"
        f"Input: {input_text}\n"
        f"Clues: {clues}\n"
        f"Reasoning:"
    )

    response_reasoning = openai.Completion.create(
        model=model,
        prompt=reasoning_prompt,
        max_tokens=130,
        temperature=0.7
    )

    reasoning = response_reasoning.choices[0].text.strip()
    print("Reasoning Response:", response_reasoning.choices[0].text)  # Print Reasoning Response

    # Step 3: Final Sentiment Determination
    sentiment_prompt = (
        f"Considering the clues and reasoning, determine the overall SENTIMENT of the input as Positive, "
        f"or Negative.\n"
        f"Input: {input_text}\n"
        f"Clues: {clues}\n"
        f"Reasoning: {reasoning}\n"
        f"Sentiment:"
    )

    response = openai.Completion.create(
        model=model,
        prompt=sentiment_prompt,
        max_tokens=10,
        temperature=0.0
    )

    print("Sentiment Determination Response:", response.choices[0].text)  # Print Sentiment Response

    # Process the sentiment result
    try:
        response = response.choices[0].text
        sentiment = response.lower().split("sentiment:")[-1].strip()

        if "neg" in sentiment:
            sentiment = 0
        elif "pos" in sentiment:
            sentiment = 1
        else:
            print(response)
            sentiment = None
    except Exception as e:
        print(f"Error processing sentiment: {e}")
        print(response)
        sentiment = None

    return sentiment


# Example usage
input_text = "its mysteries are transparently obvious , and it 's too slowly paced to be a thriller"
result = classify_text_improved(input_text)

print("Final Sentiment:", result)


Clues Response:  mysteries, transparently obvious, slowly paced, thriller
Reasoning Response: 
The input mentions that the subject matter at hand is "mysteries," and that these mysteries are "transparently obvious." This suggests that the content of the input may be easily understood or solved. Additionally, the input states that the subject matter is "too slowly paced to be a thriller." This implies that the content is not fast-paced or action-packed, which are typical characteristics of a thriller genre. Therefore, through the clues provided, it can be deduced that the input is likely not a thriller, but rather a genre that involves mysteries that are easy to understand. 
Sentiment Determination Response:  Negative
Final Sentiment: 0
